# 1. SETTINGS

In [1]:
# libraries
import pandas as pd
import numpy as np
import scipy.stats
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, train_test_split
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [2]:
# pandas options
pd.set_option("display.max_columns", None)

In [3]:
# garbage collection
import gc
gc.enable()

In [4]:
# ignore warnings
warnings.filterwarnings("ignore")

In [5]:
# random settings
seed = 42

# 2. PREPARATIONS

In [6]:
# import data
train = pd.read_csv("../data/prepared/train_full.csv")
test  = pd.read_csv("../data/prepared/test_full.csv")
y     = pd.read_csv("../data/prepared/y_full.csv")

In [7]:
# check train
train.head()

,SK_ID_CURR,app_NAME_CONTRACT_TYPE,app_CODE_GENDER,app_FLAG_OWN_CAR,app_FLAG_OWN_REALTY,app_CNT_CHILDREN,app_AMT_INCOME_TOTAL,app_AMT_CREDIT,app_AMT_ANNUITY,app_AMT_GOODS_PRICE,app_NAME_TYPE_SUITE,app_NAME_INCOME_TYPE,app_NAME_EDUCATION_TYPE,app_NAME_FAMILY_STATUS,app_NAME_HOUSING_TYPE,app_REGION_POPULATION_RELATIVE,app_DAYS_BIRTH,app_DAYS_EMPLOYED,app_DAYS_REGISTRATION,app_DAYS_ID_PUBLISH,app_OWN_CAR_AGE,app_FLAG_MOBIL,app_FLAG_EMP_PHONE,app_FLAG_WORK_PHONE,app_FLAG_CONT_MOBILE,app_FLAG_PHONE,app_FLAG_EMAIL,app_OCCUPATION_TYPE,app_CNT_FAM_MEMBERS,app_REGION_RATING_CLIENT,app_REGION_RATING_CLIENT_W_CITY,app_WEEKDAY_APPR_PROCESS_START,app_HOUR_APPR_PROCESS_START,app_REG_REGION_NOT_LIVE_REGION,app_REG_REGION_NOT_WORK_REGION,app_LIVE_REGION_NOT_WORK_REGION,app_REG_CITY_NOT_LIVE_CITY,app_REG_CITY_NOT_WORK_CITY,app_LIVE_CITY_NOT_WORK_CITY,app_ORGANIZATION_TYPE,app_EXT_SOURCE_1,app_EXT_SOURCE_2,app_EXT_SOURCE_3,app_APARTMENTS_AVG,app_BASEMENTAREA_AVG,app_YEARS_BEGINEXPLUATATION_AVG,app_YEARS_BUILD_AVG,app_COMMONAREA_AVG,app_ELEVATORS_AVG,app_ENTRANCES_AVG,app_FLOORSMAX_AVG,app_FLOORSMIN_AVG,app_LANDAREA_AVG,app_LIVINGAPARTMENTS_AVG,app_LIVINGAREA_AVG,app_NONLIVINGAPARTMENTS_AVG,app_NONLIVINGAREA_AVG,app_APARTMENTS_MODE,app_BASEMENTAREA_MODE,app_YEARS_BEGINEXPLUATATION_MODE,app_YEARS_BUILD_MODE,app_COMMONAREA_MODE,app_ELEVATORS_MODE,app_ENTRANCES_MODE,app_FLOORSMAX_MODE,app_FLOORSMIN_MODE,app_LANDAREA_MODE,app_LIVINGAPARTMENTS_MODE,app_LIVINGAREA_MODE,app_NONLIVINGAPARTMENTS_MODE,app_NONLIVINGAREA_MODE,app_APARTMENTS_MEDI,app_BASEMENTAREA_MEDI,app_YEARS_BEGINEXPLUATATION_MEDI,app_YEARS_BUILD_MEDI,app_COMMONAREA_MEDI,app_ELEVATORS_MEDI,app_ENTRANCES_MEDI,app_FLOORSMAX_MEDI,app_FLOORSMIN_MEDI,app_LANDAREA_MEDI,app_LIVINGAPARTMENTS_MEDI,app_LIVINGAREA_MEDI,app_NONLIVINGAPARTMENTS_MEDI,app_NONLIVINGAREA_MEDI,app_FONDKAPREMONT_MODE,app_HOUSETYPE_MODE,app_TOTALAREA_MODE,app_WALLSMATERIAL_MODE,app_EMERGENCYSTATE_MODE,app_OBS_30_CNT_SOCIAL_CIRCLE,app_DEF_30_CNT_SOCIAL_CIRCLE,app_OBS_60_CNT_SOCIAL_CIRCLE,app_DEF_60_CNT_SOCIAL_CIRCLE,app_DAYS_LAST_PHONE_CHANGE,app_FLAG_DOCUMENT_2,app_FLAG_DOCUMENT_3,app_FLAG_DOCUMENT_4,app_FLAG_DOCUMENT_5,app_FLAG_DOCUMENT_6,app_FLAG_DOCUMENT_7,app_FLAG_DOCUMENT_8,app_FLAG_DOCUMENT_9,app_FLAG_DOCUMENT_10,app_FLAG_DOCUMENT_11,app_FLAG_DOCUMENT_12,app_FLAG_DOCUMENT_13,app_FLAG_DOCUMENT_14,app_FLAG_DOCUMENT_15,app_FLAG_DOCUMENT_16,app_FLAG_DOCUMENT_17,app_FLAG_DOCUMENT_18,app_FLAG_DOCUMENT_19,app_FLAG_DOCUMENT_20,app_FLAG_DOCUMENT_21,app_AMT_REQ_CREDIT_BUREAU_HOUR,app_AMT_REQ_CREDIT_BUREAU_DAY,app_AMT_REQ_CREDIT_BUREAU_WEEK,app_AMT_REQ_CREDIT_BUREAU_MON,app_AMT_REQ_CREDIT_BUREAU_QRT,app_AMT_REQ_CREDIT_BUREAU_YEAR,app_CREDIT_BY_INCOME,app_ANNUITY_BY_INCOME,app_GOODS_PRICE_BY_INCOME,app_PERCENT_WORKED,app_CNT_ADULTS,app_NUM_EXT_SOURCES,app_NUM_DOCUMENTS,app_DAY_APPR_PROCESS_START,app_isnull_HOUSE,app_isnull_BURO_ENQUIRIES,app_isnull_SOCIAL_CIRCLE,buro_DAYS_END_DIFF_2_mean,buro_DAYS_END_DIFF_2_std,buro_DAYS_END_DIFF_2_min,buro_DAYS_END_DIFF_2_max,buro_DAYS_END_DIFF_1_mean,buro_DAYS_END_DIFF_1_std,buro_DAYS_END_DIFF_1_min,buro_DAYS_END_DIFF_1_max,buro_DAYS_CREDIT_mean,buro_DAYS_CREDIT_std,buro_DAYS_CREDIT_min,buro_DAYS_CREDIT_max,buro_AMT_SUM_OVERDUE_RATIO_2_mean,buro_AMT_SUM_OVERDUE_RATIO_2_std,buro_AMT_SUM_OVERDUE_RATIO_2_min,buro_AMT_SUM_OVERDUE_RATIO_2_max,buro_CNT_BURO_ACTIVE_mean,buro_CNT_BURO_ACTIVE_std,buro_CNT_BURO_ACTIVE_min,buro_CNT_BURO_ACTIVE_max,buro_CNT_BURO_CLOSED_mean,buro_CNT_BURO_CLOSED_std,buro_CNT_BURO_CLOSED_min,buro_CNT_BURO_CLOSED_max,buro_MONTH_COUNT_mean,buro_MONTH_COUNT_std,buro_MONTH_COUNT_min,buro_MONTH_COUNT_max,buro_CNT_BURO_BAD_mean,buro_CNT_BURO_BAD_std,buro_CNT_BURO_BAD_min,buro_CNT_BURO_BAD_max,buro_isnull_AMT_ANNUITY_mean,buro_isnull_AMT_ANNUITY_std,buro_isnull_AMT_ANNUITY_min,buro_isnull_AMT_ANNUITY_max,buro_DAYS_DURATION_2_mean,buro_DAYS_DURATION_2_std,buro_DAYS_DURATION_2_min,buro_DAYS_DURATION_2_max,buro_AMT_SUM_OVERDUE_RATIO_1_mean,buro_AMT_SUM_OVERDUE_RATIO_1_std,buro_AMT_SUM

In [8]:
# check test
test.head()

,SK_ID_CURR,app_NAME_CONTRACT_TYPE,app_CODE_GENDER,app_FLAG_OWN_CAR,app_FLAG_OWN_REALTY,app_CNT_CHILDREN,app_AMT_INCOME_TOTAL,app_AMT_CREDIT,app_AMT_ANNUITY,app_AMT_GOODS_PRICE,app_NAME_TYPE_SUITE,app_NAME_INCOME_TYPE,app_NAME_EDUCATION_TYPE,app_NAME_FAMILY_STATUS,app_NAME_HOUSING_TYPE,app_REGION_POPULATION_RELATIVE,app_DAYS_BIRTH,app_DAYS_EMPLOYED,app_DAYS_REGISTRATION,app_DAYS_ID_PUBLISH,app_OWN_CAR_AGE,app_FLAG_MOBIL,app_FLAG_EMP_PHONE,app_FLAG_WORK_PHONE,app_FLAG_CONT_MOBILE,app_FLAG_PHONE,app_FLAG_EMAIL,app_OCCUPATION_TYPE,app_CNT_FAM_MEMBERS,app_REGION_RATING_CLIENT,app_REGION_RATING_CLIENT_W_CITY,app_WEEKDAY_APPR_PROCESS_START,app_HOUR_APPR_PROCESS_START,app_REG_REGION_NOT_LIVE_REGION,app_REG_REGION_NOT_WORK_REGION,app_LIVE_REGION_NOT_WORK_REGION,app_REG_CITY_NOT_LIVE_CITY,app_REG_CITY_NOT_WORK_CITY,app_LIVE_CITY_NOT_WORK_CITY,app_ORGANIZATION_TYPE,app_EXT_SOURCE_1,app_EXT_SOURCE_2,app_EXT_SOURCE_3,app_APARTMENTS_AVG,app_BASEMENTAREA_AVG,app_YEARS_BEGINEXPLUATATION_AVG,app_YEARS_BUILD_AVG,app_COMMONAREA_AVG,app_ELEVATORS_AVG,app_ENTRANCES_AVG,app_FLOORSMAX_AVG,app_FLOORSMIN_AVG,app_LANDAREA_AVG,app_LIVINGAPARTMENTS_AVG,app_LIVINGAREA_AVG,app_NONLIVINGAPARTMENTS_AVG,app_NONLIVINGAREA_AVG,app_APARTMENTS_MODE,app_BASEMENTAREA_MODE,app_YEARS_BEGINEXPLUATATION_MODE,app_YEARS_BUILD_MODE,app_COMMONAREA_MODE,app_ELEVATORS_MODE,app_ENTRANCES_MODE,app_FLOORSMAX_MODE,app_FLOORSMIN_MODE,app_LANDAREA_MODE,app_LIVINGAPARTMENTS_MODE,app_LIVINGAREA_MODE,app_NONLIVINGAPARTMENTS_MODE,app_NONLIVINGAREA_MODE,app_APARTMENTS_MEDI,app_BASEMENTAREA_MEDI,app_YEARS_BEGINEXPLUATATION_MEDI,app_YEARS_BUILD_MEDI,app_COMMONAREA_MEDI,app_ELEVATORS_MEDI,app_ENTRANCES_MEDI,app_FLOORSMAX_MEDI,app_FLOORSMIN_MEDI,app_LANDAREA_MEDI,app_LIVINGAPARTMENTS_MEDI,app_LIVINGAREA_MEDI,app_NONLIVINGAPARTMENTS_MEDI,app_NONLIVINGAREA_MEDI,app_FONDKAPREMONT_MODE,app_HOUSETYPE_MODE,app_TOTALAREA_MODE,app_WALLSMATERIAL_MODE,app_EMERGENCYSTATE_MODE,app_OBS_30_CNT_SOCIAL_CIRCLE,app_DEF_30_CNT_SOCIAL_CIRCLE,app_OBS_60_CNT_SOCIAL_CIRCLE,app_DEF_60_CNT_SOCIAL_CIRCLE,app_DAYS_LAST_PHONE_CHANGE,app_FLAG_DOCUMENT_2,app_FLAG_DOCUMENT_3,app_FLAG_DOCUMENT_4,app_FLAG_DOCUMENT_5,app_FLAG_DOCUMENT_6,app_FLAG_DOCUMENT_7,app_FLAG_DOCUMENT_8,app_FLAG_DOCUMENT_9,app_FLAG_DOCUMENT_10,app_FLAG_DOCUMENT_11,app_FLAG_DOCUMENT_12,app_FLAG_DOCUMENT_13,app_FLAG_DOCUMENT_14,app_FLAG_DOCUMENT_15,app_FLAG_DOCUMENT_16,app_FLAG_DOCUMENT_17,app_FLAG_DOCUMENT_18,app_FLAG_DOCUMENT_19,app_FLAG_DOCUMENT_20,app_FLAG_DOCUMENT_21,app_AMT_REQ_CREDIT_BUREAU_HOUR,app_AMT_REQ_CREDIT_BUREAU_DAY,app_AMT_REQ_CREDIT_BUREAU_WEEK,app_AMT_REQ_CREDIT_BUREAU_MON,app_AMT_REQ_CREDIT_BUREAU_QRT,app_AMT_REQ_CREDIT_BUREAU_YEAR,app_CREDIT_BY_INCOME,app_ANNUITY_BY_INCOME,app_GOODS_PRICE_BY_INCOME,app_PERCENT_WORKED,app_CNT_ADULTS,app_NUM_EXT_SOURCES,app_NUM_DOCUMENTS,app_DAY_APPR_PROCESS_START,app_isnull_HOUSE,app_isnull_BURO_ENQUIRIES,app_isnull_SOCIAL_CIRCLE,buro_DAYS_END_DIFF_2_mean,buro_DAYS_END_DIFF_2_std,buro_DAYS_END_DIFF_2_min,buro_DAYS_END_DIFF_2_max,buro_DAYS_END_DIFF_1_mean,buro_DAYS_END_DIFF_1_std,buro_DAYS_END_DIFF_1_min,buro_DAYS_END_DIFF_1_max,buro_DAYS_CREDIT_mean,buro_DAYS_CREDIT_std,buro_DAYS_CREDIT_min,buro_DAYS_CREDIT_max,buro_AMT_SUM_OVERDUE_RATIO_2_mean,buro_AMT_SUM_OVERDUE_RATIO_2_std,buro_AMT_SUM_OVERDUE_RATIO_2_min,buro_AMT_SUM_OVERDUE_RATIO_2_max,buro_CNT_BURO_ACTIVE_mean,buro_CNT_BURO_ACTIVE_std,buro_CNT_BURO_ACTIVE_min,buro_CNT_BURO_ACTIVE_max,buro_CNT_BURO_CLOSED_mean,buro_CNT_BURO_CLOSED_std,buro_CNT_BURO_CLOSED_min,buro_CNT_BURO_CLOSED_max,buro_MONTH_COUNT_mean,buro_MONTH_COUNT_std,buro_MONTH_COUNT_min,buro_MONTH_COUNT_max,buro_CNT_BURO_BAD_mean,buro_CNT_BURO_BAD_std,buro_CNT_BURO_BAD_min,buro_CNT_BURO_BAD_max,buro_isnull_AMT_ANNUITY_mean,buro_isnull_AMT_ANNUITY_std,buro_isnull_AMT_ANNUITY_min,buro_isnull_AMT_ANNUITY_max,buro_DAYS_DURATION_2_mean,buro_DAYS_DURATION_2_std,buro_DAYS_DURATION_2_min,buro_DAYS_DURATION_2_max,buro_AMT_SUM_OVERDUE_RATIO_1_mean,buro_AMT_SUM_OVERDUE_RATIO_1_std,buro_AMT_SUM

In [9]:
# extract target
y = y["TARGET"]

In [10]:
# exclude features
excluded_feats = ["SK_ID_CURR"]
features = [f for f in train.columns if f not in excluded_feats]

In [11]:
# check dimensions
print(train[features].shape)
print(test[features].shape)

(307511, 1072)
(48744, 1072)


In [23]:
### PARAMETERS

# parallel settings
cores = 16

# learner settings
metric   = "auc"
verbose  = 250
stopping = 300

# CV settings
num_folds = 5
shuffle   = True

# lightGBM
gbm = lgb.LGBMClassifier(n_estimators     = 10000,
                         learning_rate    = 0.005,
                         num_leaves       = 70,
                         colsample_bytree = 0.8,
                         subsample        = 0.9,
                         max_depth        = 7,
                         reg_alpha        = 0.1,
                         reg_lambda       = 0.1,
                         min_split_gain   = 0.01,
                         min_child_weight = 2,
                         random_state     = seed,
                         num_threads      = cores)

# 3. CROSS-VALIDATION

In [24]:
# data partitinoing
folds = StratifiedKFold(n_splits = num_folds, random_state = seed, shuffle = shuffle)

# placeholders
valid_aucs_cv = np.zeros(num_folds) 
test_preds_cv = np.zeros(test.shape[0])

In [25]:
### CROSS-VALIDATION LOOP
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train, y)):
    
    # data partitioning
    trn_x, trn_y = train[features].iloc[trn_idx], y.iloc[trn_idx]
    val_x, val_y = train[features].iloc[val_idx], y.iloc[val_idx]
    
    # train lightGBM
    gbm = gbm.fit(trn_x, trn_y, 
                  eval_set = [(trn_x, trn_y), (val_x, val_y)], 
                  eval_metric = metric, 
                  verbose = verbose, 
                  early_stopping_rounds = stopping)
    
    # save number of iterations
    num_iter_cv = gbm.best_iteration_
    
    # predictions
    valid_preds_cv = gbm.predict_proba(val_x, num_iteration = num_iter_cv)[:, 1]
    valid_aucs_cv[n_fold] = roc_auc_score(val_y, valid_preds_cv)
    test_preds_cv += gbm.predict_proba(test[features], num_iteration = num_iter_cv)[:, 1] / folds.n_splits
    
    # print performance
    print("----------------------")
    print("Fold%2d AUC: %.6f" % (n_fold + 1, valid_aucs_cv[n_fold]))
    print("----------------------")
    print("")

    # clear memory
    del trn_x, trn_y, val_x, val_y
    gc.collect()
    
# print overall performance    
print("Cross-Validation AUC score %.6f" % np.mean(valid_aucs_cv))

# RESULTS   
# no card, old features (560):    0.786941 | 0.783
# no card, new features (694):    0.788893 | 0.783
# with card, new features (1072): 0.

Training until validation scores don't improve for 300 rounds.
[250]	training's auc: 0.770236	valid_1's auc: 0.746511
[500]	training's auc: 0.786326	valid_1's auc: 0.754542
[750]	training's auc: 0.805473	valid_1's auc: 0.763446
[1000]	training's auc: 0.823347	valid_1's auc: 0.770695
[1250]	training's auc: 0.837965	valid_1's auc: 0.775669
[1500]	training's auc: 0.850356	valid_1's auc: 0.778992
[1750]	training's auc: 0.861232	valid_1's auc: 0.781242
[2000]	training's auc: 0.870929	valid_1's auc: 0.782938
[2250]	training's auc: 0.879507	valid_1's auc: 0.784204
[2500]	training's auc: 0.887496	valid_1's auc: 0.785294
[2750]	training's auc: 0.894626	valid_1's auc: 0.786128
[3000]	training's auc: 0.901016	valid_1's auc: 0.786839
[3250]	training's auc: 0.906839	valid_1's auc: 0.787413
[3500]	training's auc: 0.912411	valid_1's auc: 0.787931
[3750]	training's auc: 0.917517	valid_1's auc: 0.788266
[4000]	training's auc: 0.922296	valid_1's auc: 0.788678
[4250]	training's auc: 0.926698	valid_1's au

MemoryError: 

# 3. SUBMISSION

In [15]:
# create submission
test["TARGET"] = test_preds_cv
subm = test[["SK_ID_CURR", "TARGET"]]

# export CSV
subm.to_csv("../submissions/bagged_lgb_10k_all1072.csv", index = False, float_format = "%.8f")